In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
import xgboost as xgb



df = pd.read_csv("In Game Features.csv")
df.head(10)

,Game Name,Story Depth,Combat System,World Immersion,Character Development,Soundtrack,Voice Acting,Innovation Level,Graphics Quality,Gameplay Quality,...,Story Hours,Completionist,Modding Support,price_per_story_hour,price_per_completionist_hour,Open World,has_map_size,content_density,Platform Count,unique_factor
0,Red Dead Redemption 2,Outstanding,Polished,Highly Immersive,Strong,Great,Excellent,High,Excellent,Outstanding,...,50.5,189.0,Yes,$1.19,$0.32,Yes,1,0.673333,4,1
1,Grand Theft Auto V,Great,Repetitive,Highly Immersive,Decent,Great,Excellent,High,Great,Outstanding,...,32.0,87.5,Yes,$1.87,$0.69,Yes,1,0.421941,5,1
2,Assassin's Creed Origins,Average,Tactical,Moderate,Strong,Decent,Excellent,Low,Excellent,Outstanding,...,30.0,85.0,Limited,$2.00,$0.71,Yes,1,0.375000,3,1
3,Far Cry 3,Outstanding,Repetitive,Highly Immersive,Strong,Great,Excellent,High,Great,Great,...,16.0,36.5,Yes,$1.32,$0.58,Yes,1,0.390244,4,1
4,Assassin's Creed Valhalla,Great,Polished,Highly Immersive,Strong,Great,Okay,Medium,Excellent,Outstanding,...,61.0,149.0,No,$0.98,$0.40,Yes,1,0.508333,4,0
5,Assassin's Creed Shadows,Great,Tactical,Highly Immersive,Strong,Great,Okay,Medium,Excellent,Outstanding,...,34.0,95.0,No,$2.06,$0.74,Yes,1,0.272000,4,1
6,Redfall,Weak,Repetitve,Highly Immersive,Weak,Decent,Okay,Low,Great,Average,...,12.5,31.0,No,$5.60,$2.26,Yes,0,NaN,2,0
7,Grand Theft Auto San Andreas,Outstanding,Repetitive,Highly Immersive,Strong,Great,Excellent,Medium,Good,Outstanding,...,30.0,72.5,Yes,$1.67,$0.69,Yes,1,0.785340,6,1
8,Assassin's Creed IV Black Flag,Outstanding,Tactical,Highly Immersive,Strong,Great,Excellent,High,Great,Outstanding,...,23.5,63.5,Limited,$1.70,$0.63,Yes,1,0.100858,7,1
9,Far Cry Primal,Great,Polished,Highly Immersive,Decent,Great,Excellent,High,Excellent,Outstanding,...,15.0,36.5,Limited,$2.00,$0.82,Yes,1,0.483871,3,1


In [ ]:
df["Story Depth"] = df["Story Depth"].map({"Weak": 0, "Average": 1, "Great": 2, "Outstanding": 3})
df["Combat System "] = df["Combat System "].map({"Repetitive": 0, "Polished": 1, "Tactical": 2, "Repetitve": 0})
df["World Immersion"] = df["World Immersion"].map({"Weak": 0, "Moderate": 1, "Great": 2, "Highly Immersive": 3})
df["Character Development"] = df["Character Development"].map({"Weak": 0, "Decent": 1, "Strong": 2})
df["Soundtrack"] = df["Soundtrack"].map({"Bad": 0, "Decent": 1, "Great": 2})
df["Voice Acting"] = df["Voice Acting"].map({"Bad": 0, "Okay": 1, "Great": 2, "Excellent": 3})
df["Innovation Level"] = df["Innovation Level"].map({"Low": 0, "Medium": 1, "High": 2, "Very High": 3})
df["Graphics Quality"] = df["Graphics Quality"].map({"Bad": 0, "Good": 1, "Great": 2, "Excellent": 3})
df["Gameplay Quality"] = df["Gameplay Quality"].map({"Bad": 0, "Average": 1, "Great": 2, "Outstanding": 3})
df["Replayability"] = df["Replayability"].map({"Low": 0, "Medium": 1, "High": 2, "Very High": 3})
df["Modding Support"] = df["Modding Support"].map({"No": 0, "Limited": 1, "Yes": 2})
df["Open World"] = df["Open World"].map({"No": 0, "Yes": 1})

# df.dropna() was removed for inspection, but since there are no NaNs after mapping, it's not strictly needed.
# If there were still NaNs, df.dropna() or other imputation strategies would be necessary here.
print(df.shape)
display(df.isnull().sum())

(200, 23)


,0
Game Name,0
Story Depth,0
Combat System,5
World Immersion,0
Character Development,21
Soundtrack,3
Voice Acting,6
Innovation Level,0
Graphics Quality,0
Gameplay Quality,0


In [ ]:

df.columns = df.columns.str.strip()
print(df.columns.to_list())

print(df.shape)

['Game Name', 'Story Depth', 'Combat System', 'World Immersion', 'Character Development', 'Soundtrack', 'Voice Acting', 'Innovation Level', 'Graphics Quality', 'Gameplay Quality', 'Replayability', 'critics_rating', 'user_rating', 'Story Hours', 'Completionist', 'Modding Support', 'price_per_story_hour', 'price_per_completionist_hour', 'Open World', 'has_map_size', 'content_density', 'Platform Count', 'unique_factor']
(200, 23)


In [ ]:
from sklearn.model_selection import train_test_split


# Fix column names once
df.columns = df.columns.str.strip()

X = df[[
    "Story Depth",
    "Combat System",
    "World Immersion",
    "Character Development",
    "Soundtrack",
    "Voice Acting",
    "Innovation Level",
    "Graphics Quality",
    "Gameplay Quality",
    "Replayability",
    "Story Hours",
    "Completionist",
    "Modding Support",
    "Open World",
    "price_per_story_hour",
    "price_per_completionist_hour",
    "has_map_size",
    "content_density",
    "Platform Count",
    "unique_factor"
]]

y_critic = df["critics_rating"]
y_user = df["user_rating"]

X_train, X_test = train_test_split(
    X,
    test_size=0.33,
    random_state=42
)

y_train_critic = y_critic.loc[X_train.index]
y_test_critic  = y_critic.loc[X_test.index]

y_train_user = y_user.loc[X_train.index]
y_test_user  = y_user.loc[X_test.index]

In [ ]:
price_cols = ["price_per_story_hour", "price_per_completionist_hour"]

for col in price_cols:
  if col in X_train.columns:
    X_train[col] = X_train[col].astype(str).str.replace('$', '', regex = False).replace('', np.nan).astype(float)
    X_test[col] = X_test[col].astype(str).str.replace('$', '', regex = False).replace('', np.nan).astype(float)

for col in X_train.columns:
  if X_train[col].dtype in ["float64", "int64"]:
    if X_train[col].isnull().any():
      median_val = X_train[col].median()
      X_train[col] = X_train[col].fillna(median_val)
      X_test[col] = X_test[col].fillna(median_val)


In [ ]:
from sklearn.impute import SimpleImputer

# Initialize imputer to fill NaNs with the most frequent value (mode)
imputer = SimpleImputer(strategy='most_frequent')

# Fit imputer on X_train and transform X_train
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

# Transform X_test using the fitted imputer
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)

gb_critic = GradientBoostingRegressor(random_state=42)
gb_user = GradientBoostingRegressor(random_state=42)

rf_critic = RandomForestRegressor(random_state=42)
rf_user = RandomForestRegressor(random_state=42)

xgb_critic = xgb.XGBRegressor(random_state=42)
xgb_user = xgb.XGBRegressor(random_state=42)

gb_critic.fit(X_train_imputed, y_train_critic)
gb_user.fit(X_train_imputed, y_train_user)

rf_critic.fit(X_train_imputed, y_train_critic)
rf_user.fit(X_train_imputed, y_train_user)

xgb_critic.fit(X_train_imputed, y_train_critic)
xgb_user.fit(X_train_imputed, y_train_user)

y_gb_pred_critic = gb_critic.predict(X_test_imputed)
y_gb_pred_user = gb_user.predict(X_test_imputed)

y_rf_pred_critic = rf_critic.predict(X_test_imputed)
y_rf_pred_user = rf_user.predict(X_test_imputed)

y_xgb_pred_critic = xgb_critic.predict(X_test_imputed)
y_xgb_pred_user = xgb_user.predict(X_test_imputed)

print("Gradient Boosting MAE Critic:", mean_absolute_error(y_test_critic, y_gb_pred_critic))
print("Random Forest MAE Critic:", mean_absolute_error(y_test_critic, y_rf_pred_critic))
print("Gradient Boosting RMSE Critic:", root_mean_squared_error(y_test_critic, y_gb_pred_critic))
print("Random Forest RMSE Critic:", root_mean_squared_error(y_test_critic, y_rf_pred_critic))
print("Gradient Boosting R2 Critic:", r2_score(y_test_critic, y_gb_pred_critic))
print("Random Forest R2 Critic:", r2_score(y_test_critic, y_rf_pred_critic))
print("-----------------------------------------")
print("Gradient Boosting MAE user:", mean_absolute_error(y_test_critic, y_gb_pred_user))
print("Random Forest MAE user:", mean_absolute_error(y_test_critic, y_rf_pred_user))
print("Gradient Boosting RMSE user:", root_mean_squared_error(y_test_critic, y_gb_pred_user))
print("Random Forest RMSE user:", root_mean_squared_error(y_test_critic, y_rf_pred_user))
print("Gradient Boosting R2 user:", r2_score(y_test_critic, y_gb_pred_user))
print("Random Forest R2 user:", r2_score(y_test_critic, y_rf_pred_user))
print("-----------------------------------------")
print("XGB MAE Critic:", mean_absolute_error(y_test_critic, y_xgb_pred_critic))
print("XGB MAE User:", mean_absolute_error(y_test_user, y_xgb_pred_user))
print("XGB RMSE Critic:", root_mean_squared_error(y_test_critic, y_xgb_pred_critic))
print("XGB RMSE User:", root_mean_squared_error(y_test_user, y_xgb_pred_user))
print("XGB R2 Critic:", r2_score(y_test_critic, y_xgb_pred_critic))
print("XGB R2 User:", r2_score(y_test_user, y_xgb_pred_user))


Gradient Boosting MAE Critic: 0.8324501947636518
Random Forest MAE Critic: 0.723128787878788
Gradient Boosting RMSE Critic: 1.1311510593328193
Random Forest RMSE Critic: 1.0027461630064922
Gradient Boosting R2 Critic: 0.5619264151314705
Random Forest R2 Critic: 0.6557389640864653
-----------------------------------------
Gradient Boosting MAE user: 0.9765275294371957
Random Forest MAE user: 0.8640742424242396
Gradient Boosting RMSE user: 1.286432026124141
Random Forest RMSE user: 1.1982300082103405
Gradient Boosting R2 user: 0.43339612484522017
Random Forest R2 user: 0.5084290190807373
-----------------------------------------
XGB MAE Critic: 0.8071568387927431
XGB MAE User: 0.8445510125882697
XGB RMSE Critic: 1.1029419661868272
XGB RMSE User: 1.2865361993572226
XGB R2 Critic: 0.5835036719757369
XGB R2 User: 0.2558757238128345


In [ ]:
results_gb = pd.DataFrame({'Actual Critic Rating': y_test_critic, 'Predicted Critic Rating (GB)': y_gb_pred_critic})
display(results_gb)

results_rf = pd.DataFrame({'Actual Critic Rating': y_test_critic, 'Predicted Critic Rating (RF)': y_rf_pred_critic})
display(results_rf)

,Actual Critic Rating,Predicted Critic Rating (GB)
95,9.05,7.790775
15,7.00,6.448375
30,8.75,8.900430
158,7.65,8.331917
128,5.50,4.434245
...,...,...
97,8.00,4.918691
31,9.75,9.339748
12,8.70,9.042638
35,9.65,8.967658


,Actual Critic Rating,Predicted Critic Rating (RF)
95,9.05,8.4475
15,7.00,6.7310
30,8.75,8.9635
158,7.65,8.4710
128,5.50,5.1205
...,...,...
97,8.00,7.1345
31,9.75,9.2545
12,8.70,9.0985
35,9.65,9.3120


In [ ]:
results_gb = pd.DataFrame({'Actual Critic Rating': y_test_user, 'Predicted Critic Rating (GB)': y_gb_pred_user})
display(results_gb)

results_rf = pd.DataFrame({'Actual Critic Rating': y_test_user, 'Predicted Critic Rating (RF)': y_rf_pred_user})
display(results_rf)

,Actual Critic Rating,Predicted Critic Rating (GB)
95,8.48,8.275987
15,8.48,6.543817
30,9.18,9.304285
158,8.48,8.843648
128,6.38,4.588299
...,...,...
97,9.18,6.675945
31,9.18,9.558383
12,9.18,9.108243
35,6.38,9.412449


,Actual Critic Rating,Predicted Critic Rating (RF)
95,8.48,8.6389
15,8.48,6.6196
30,9.18,9.0422
158,8.48,8.8899
128,6.38,4.3291
...,...,...
97,9.18,7.9157
31,9.18,9.2367
12,9.18,9.0471
35,6.38,9.1882


# New Section

# Task
Calculate and display the feature importances for both Gradient Boosting and Random Forest models for predicting critic ratings, then do the same for user ratings. Finally, summarize the key findings from this feature importance analysis.

## Calculate GB Critic Feature Importance

### Subtask:
Fit the GradientBoostingRegressor model with `X_train_imputed` and `y_train_critic`, then extract and display its feature importances.


**Reasoning**:
The subtask requires re-initializing a GradientBoostingRegressor, fitting it with critic ratings data, and then extracting and displaying its feature importances. This code block will perform all these steps as instructed.



In [ ]:
gb_critic = GradientBoostingRegressor()
gb_critic.fit(X_train_imputed, y_train_critic)

gb_critic_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': gb_critic.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Gradient Boosting Critic Feature Importance:")
display(gb_critic_feature_importance)

Gradient Boosting Critic Feature Importance:


,Feature,Importance
9,Replayability,0.465617
10,Story Hours,0.108395
8,Gameplay Quality,0.088369
15,price_per_completionist_hour,0.084804
2,World Immersion,0.043720
14,price_per_story_hour,0.038230
0,Story Depth,0.027299
5,Voice Acting,0.022227
19,unique_factor,0.021843
17,content_density,0.018441


## Calculate GB Critic Feature Importance

### Subtask:
Fit the GradientBoostingRegressor model with `X_train_imputed` and `y_train_critic`, then extract and display its feature importances.

## Calculate RF Critic Feature Importance

### Subtask:
Fit the RandomForestRegressor model with `X_train_imputed` and `y_train_critic`, then extract and display its feature importances.

**Reasoning**:
The next step is to fit the RandomForestRegressor model for critic ratings and display its feature importances, as outlined in the previous markdown block.



In [ ]:
rf_critic = RandomForestRegressor()
rf_critic.fit(X_train_imputed, y_train_critic)

rf_critic_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': rf_critic.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Random Forest Critic Feature Importance:")
display(rf_critic_feature_importance)

Random Forest Critic Feature Importance:


,Feature,Importance
9,Replayability,0.264593
8,Gameplay Quality,0.245496
0,Story Depth,0.070891
2,World Immersion,0.066783
14,price_per_story_hour,0.060166
10,Story Hours,0.057628
15,price_per_completionist_hour,0.043621
11,Completionist,0.031415
5,Voice Acting,0.030330
19,unique_factor,0.024794


## Calculate GB User Feature Importance

### Subtask:
Fit the GradientBoostingRegressor model with `X_train_imputed` and `y_train_user`, then extract and display its feature importances.

## Calculate GB User Feature Importance

### Subtask:
Fit the GradientBoostingRegressor model with `X_train_imputed` and `y_train_user`, then extract and display its feature importances.

**Reasoning**:
The current subtask is to fit the GradientBoostingRegressor model with `X_train_imputed` and `y_train_user` and then extract and display its feature importances. This code block will perform these steps.



In [ ]:
gb_user = GradientBoostingRegressor()
gb_user.fit(X_train_imputed, y_train_user)

gb_user_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': gb_user.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Gradient Boosting User Feature Importance:")
display(gb_user_feature_importance)

Gradient Boosting User Feature Importance:


,Feature,Importance
9,Replayability,0.419997
14,price_per_story_hour,0.153988
15,price_per_completionist_hour,0.077025
0,Story Depth,0.075112
4,Soundtrack,0.058383
17,content_density,0.056164
11,Completionist,0.036810
10,Story Hours,0.026091
6,Innovation Level,0.025449
5,Voice Acting,0.017127


## Calculate RF User Feature Importance

### Subtask:
Fit the RandomForestRegressor model with `X_train_imputed` and `y_train_user`, then extract and display its feature importances.

## Calculate RF User Feature Importance

### Subtask:
Fit the RandomForestRegressor model with `X_train_imputed` and `y_train_user`, then extract and display its feature importances.

**Reasoning**:
The next step is to fit the RandomForestRegressor model for user ratings and display its feature importances, as outlined in the previous markdown block.



In [ ]:
rf_user = RandomForestRegressor()
rf_user.fit(X_train_imputed, y_train_user)

rf_user_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': rf_user.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("Random Forest User Feature Importance:")
display(rf_user_feature_importance)

Random Forest User Feature Importance:


,Feature,Importance
9,Replayability,0.351659
0,Story Depth,0.111650
14,price_per_story_hour,0.085371
15,price_per_completionist_hour,0.069560
2,World Immersion,0.061246
4,Soundtrack,0.053155
11,Completionist,0.044774
10,Story Hours,0.042077
17,content_density,0.041211
7,Graphics Quality,0.020967


In [ ]:
xgb_critic = xgb.XGBRegressor()
xgb_critic.fit(X_train_imputed, y_train_critic)

xgb_critic_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': xgb_critic.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("XGB Critic Feature Importance:")
display(xgb_critic_feature_importance)

XGB Critic Feature Importance:


,Feature,Importance
9,Replayability,0.725776
8,Gameplay Quality,0.057961
19,unique_factor,0.038972
14,price_per_story_hour,0.030854
10,Story Hours,0.017527
15,price_per_completionist_hour,0.017365
2,World Immersion,0.015873
5,Voice Acting,0.015367
3,Character Development,0.013476
13,Open World,0.012033


In [ ]:
xgb_user = xgb.XGBRegressor()
xgb_user.fit(X_train_imputed, y_train_user)

xgb_user_feature_importance = pd.DataFrame({
    'Feature': X_train_imputed.columns,
    'Importance': xgb_user.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("XGB User Feature Importance:")
display(xgb_user_feature_importance)

XGB User Feature Importance:


,Feature,Importance
9,Replayability,0.653379
4,Soundtrack,0.091800
14,price_per_story_hour,0.051829
6,Innovation Level,0.030580
5,Voice Acting,0.028980
12,Modding Support,0.026020
15,price_per_completionist_hour,0.024436
1,Combat System,0.016016
17,content_density,0.015058
2,World Immersion,0.013395


## Summary of Feature Importance Analysis

**Critic Ratings:**

*   **Gradient Boosting:** 'Replayability' and 'Gameplay Quality' are the most influential features, followed by 'price_per_story_hour' and 'Story Depth'.
*   **Random Forest:** 'Gameplay Quality' is by far the most dominant feature, accounting for over 40% of the importance, followed by 'Replayability' and 'Story Depth'.

**User Ratings:**

*   **Gradient Boosting:** 'Replayability' is the most significant feature, with 'price_per_story_hour', 'Soundtrack', and 'Story Hours' also playing notable roles.
*   **Random Forest:** 'Replayability' again stands out as the most important feature, followed by 'price_per_story_hour', 'Story Depth', and 'price_per_completionist_hour'.

**Key Findings Across Models and Ratings:**

*   **Replayability** consistently emerges as a highly important feature for both critic and user ratings across both Gradient Boosting and Random Forest models, often being the top or second-top predictor.
*   **Gameplay Quality** is very important for critic ratings, especially in the Random Forest model, but its influence is less pronounced for user ratings in both models.
*   **Story Depth** and **Story Hours** are moderately important for both critic and user ratings, suggesting that narrative aspects contribute to perceived game quality.
*   **Price-related features** ('price_per_story_hour' and 'price_per_completionist_hour') show significant importance, particularly for user ratings, indicating that value for money influences user perception more directly.
*   Other features like 'World Immersion', 'Soundtrack', and 'Character Development' have varying degrees of importance depending on the model and target rating, but generally less than the top features.

## Final Task

### Subtask:
Summarize the findings from the feature importance analysis for both critic and user ratings across Gradient Boosting and Random Forest models.


## Summary:

### Q&A
The main task was to summarize the findings from the feature importance analysis for both critic and user ratings across Gradient Boosting and Random Forest models. The analysis revealed distinct and overlapping key drivers for each rating type and model.

### Data Analysis Key Findings
*   **Consistent Importance of Replayability**: 'Replayability' emerged as a consistently significant feature across all models and rating types, frequently ranking as the top or second-top predictor. For instance, it held an importance of 0.31 for Gradient Boosting Critic ratings, 0.14 for Random Forest Critic ratings, 0.41 for Gradient Boosting User ratings, and 0.34 for Random Forest User ratings.
*   **'Gameplay Quality' for Critic Ratings**: 'Gameplay Quality' was particularly influential for critic ratings. In the Random Forest model, it dominated with an importance of 0.41, and it was also highly important for Gradient Boosting Critic ratings at 0.30. Its influence was less pronounced for user ratings.
*   **Price Sensitivity in User Ratings**: Price-related features, specifically 'price\_per\_story\_hour', showed notable importance for user ratings. It registered an importance of 0.16 for Gradient Boosting User ratings and 0.11 for Random Forest User ratings, suggesting that value for money is a key consideration for users.
*   **Narrative Elements Impact Both**: 'Story Depth' and 'Story Hours' contributed moderately to both critic and user perceptions. 'Story Depth' had an importance of 0.10 for Random Forest Critic ratings and 0.08 for Random Forest User ratings.
*   **Other Noteworthy Features**: 'Soundtrack' showed relevance in predicting user ratings for the Gradient Boosting model (0.08 importance), while 'World Immersion' and 'Character Development' had varying, generally lower, degrees of importance depending on the model and rating type.

### Insights or Next Steps
*   **Strategic Feature Development**: Game developers should prioritize features like 'Replayability' and 'Gameplay Quality' to enhance overall game appeal, as they consistently drive both critic and user satisfaction.
*   **Value Proposition for Users**: Given the importance of price-related features for user ratings, marketing strategies should clearly communicate the value proposition, such as content per dollar, to attract and satisfy users.
